# Notebook to generate expected mutants

## Import libraries

In [ ]:
import pandas as pd
import itertools
import numpy as np

## Specify paths

In [ ]:
# Note, I used to import config entries as "params" (i.e. listed in the params statement of the rule), but...
# a bug led to params not being imported.. (snakemake object is imported but not the params attribute)
# Therefore, now I will import all config entries directly from the snakemake object
wtseq_path = (
    snakemake.input.wtseqs  # Projet-specific file containing the wild-type sequences
)
codon_table_path = snakemake.config["codon"][
    "table"  # Projet-specific file containing the genetic code
]
codon_mode = snakemake.config["codon"][
    "mode"  # Project-specific parameter to specify which degenerate codons were introduced
]
output_file = snakemake.output[0]

## Import codon table and codon mode

In [ ]:
if codon_mode not in ["NNN", "NNK", "NNN x NNN", "NNK x NNK"]:
    raise Exception(f"Error.. check the codon mode specified in the config.")
elif codon_mode in ["NNN", "NNK"]:
    single_codon_mode = codon_mode
else:
    single_codon_mode = codon_mode.split(" x ")[0]

In [ ]:
codon_table = pd.read_csv(codon_table_path, header=0)
codon_table["codon"] = codon_table["codon"].str.upper()
codon_table.head(3)

In [ ]:
# Convert to dictionary
codon_dic = dict(zip(codon_table["codon"], codon_table["aminoacid"]))

## Define and test functions

In [ ]:
def get_alt_codons(seq, codon_dic, mode="NNN"):
    """
    Based on a DNA sequence, the function returns two lists:
    1) A list containing all 0-based amino acid positions for the sequence
    2) A list containing all possible alternative codons (other than WT codon) at the matching positions
    For list 2, the mode defines which codons are acceptable: NNN by default, or NNK
    Codons are fetched in the provided codon table (dictionary)
    """

    if mode == "NNN":
        alt = [x for x in codon_dic.keys()]
    elif mode == "NNK":
        alt = [x for x in codon_dic.keys() if x[2] in ["G", "T"]]
    else:
        print("Pleae specify a correct mode: either NNN or NNK")

    pos_l = []
    var_l = []

    for i in range(0, len(seq), 3):
        list_var = [x for x in alt if x != seq[i : i + 3]]
        pos_l.append(i // 3)  # 0-based position (aa)
        var_l.append(list_var)  # list of possible codons other than WT

    return pos_l, var_l

In [ ]:
print(get_alt_codons("TCTCCTGTT", codon_dic, "NNN"))

In [ ]:
def get_nt_seq(seq, mut_dic):
    list_codons = [
        seq[i : i + 3]
        for i in range(0, len(seq), 3)  # Convert nucleotide sequence to list of codons
    ]

    for pos, mut_codon in mut_dic.items():
        if 0 <= pos < len(list_codons):
            list_codons[pos] = mut_codon

    return "".join(list_codons)

In [ ]:
get_nt_seq("TCTCCTGTT", {0: "TTC", 2: "TTA"})

## Import sequences

In [ ]:
wt_df = pd.read_csv(wtseq_path)
wt_df["nt_seq"] = wt_df["WT_seq"].str.upper()
wt_df

## Get single mutants

### Get all allowed codons

In [ ]:
wt_seq = wt_df.copy()
wt_seq["pos"], wt_seq["alt_codons"] = zip(
    *wt_seq.WT_seq.apply(lambda x: get_alt_codons(x, codon_dic, single_codon_mode))
)
wt_seq

### Reshape

In [ ]:
singles_compact = wt_seq.explode(["pos", "alt_codons"])
singles_compact.head(2)

In [ ]:
singles_df = singles_compact.explode("alt_codons")
singles_df

In [ ]:
singles_df["mutations"] = singles_df.apply(
    lambda row: {row[f"pos"]: row[f"alt_codons"]}, axis=1
)
singles_df

## Get double mutants (if specified in the config)

In [ ]:
if codon_mode in ["NNN x NNN", "NNK x NNK"]:
    pairwise_df = wt_df.copy()

    # Get pairwise combinations of codons that will be mutated
    pairwise_df["combination"] = pairwise_df.WT_seq.apply(
        lambda x: [
            x
            for x in list(itertools.combinations(range(0, len(x) // 3), 2))
            for _ in range(2)
        ]
    )
    doubles_compact_df = pairwise_df.explode("combination")
    doubles_compact_df["mutated_codon"] = np.tile(
        [1, 2], len(doubles_compact_df) // 2 + 1
    )[: len(doubles_compact_df)]
    doubles_compact_df["pos"] = doubles_compact_df.apply(
        lambda row: row.combination[row.mutated_codon - 1], axis=1
    )

    # Refer to a previously built dataframe to get alternative codons for each mutated codon (out of 2) at each position
    doubles_compact_df["alt_codons"] = doubles_compact_df.apply(
        lambda row: singles_compact.loc[
            (singles_compact.Mutated_seq == row.Mutated_seq)
            & (singles_compact.pos == row.pos),
            "alt_codons",
        ].item(),
        axis=1,
    )

    # Dataframe is pivoted only to be able to use pd.explode(), then later on melted to go back to long format
    doubles_piv = doubles_compact_df.pivot_table(
        index=["Mutated_seq", "WT_seq", "combination"],
        columns="mutated_codon",
        values=["pos", "alt_codons"],
        aggfunc="first",
    ).reset_index()
    doubles_piv.columns = [x[0] for x in doubles_piv.columns[:-4]] + [
        f"{x[0]}{x[1]}" for x in doubles_piv.columns[-4:]
    ]

    # Reshape
    doubles_exp1 = doubles_piv.explode("alt_codons1")
    doubles_exp2 = doubles_exp1.explode("alt_codons2")
    doubles_df = doubles_exp2.reset_index(drop=True)

    # Create dictionary of mutations, here we go with numpy to be as efficient as possible, even with very large datasets
    keys = np.stack([doubles_df["pos1"].values, doubles_df["pos2"].values], axis=1)
    vals = np.stack(
        [doubles_df["alt_codons1"].values, doubles_df["alt_codons2"].values], axis=1
    )
    doubles_df["mutations"] = [dict(zip(k, v)) for k, v in zip(keys, vals)]

else:
    doubles_df = pd.DataFrame()

## Concatenate single mutants and double mutants

In [ ]:
mutants_df = pd.concat([singles_df, doubles_df], ignore_index=True)

## Construct sequences

In [ ]:
# Again, we need an efficient way to apply our function to get the nucleotide sequences
mutants_df["nt_seq"] = list(
    itertools.starmap(get_nt_seq, zip(mutants_df["WT_seq"], mutants_df["mutations"]))
)
mutants_df

## Add wild-type nucleotide sequence

In [ ]:
# Note: information on mutated codons is dropped so that we can annotate more types of mutants in downstream steps
expmut_df = pd.concat(
    [wt_df, mutants_df.drop(["mutations", "pos", "alt_codons"], axis=1)],
    ignore_index=True,
)

## Export

In [ ]:
expmut_df.to_csv(output_file)